# CHAPTER 3 파이썬으로 만드는 투자 전략과 주요 지표
------------

* 2장의 내용을 바탕으로 Buy & Hold 전략을 만드는 실습 진행
* 투자 전략을 만들고 주요 지표를 계산해 본다.

## 3.1 바이앤홀드 전략
-----

* 특정 종목이 충분히 저가라고 판단될 때 주기적으로 매수한 후 장기간 보유하는 투자 전략
* 매수한 종목의 가격이 충분히 올랐다고 판단하면 매도
* 경제가 성장함에 따라 장기적으로 우상향한다는 투자 철학의 관점에서 보면 합리적인 투자 방법이라고 할 수 있다.

### 3.1.1 데이터 불러오기
----------

* 야후 파이낸스의 아마존 주식 다운로드
    * https://finance.yahoo.com/quote/AMZN/history?p=AMZN

```python
import pandas as pd
import numpy as np

# Date를 index로 하는 시가(Open), 고가(High), 저가(Low), 종가(Close), 수정 종가(Adj Close), 거래량(Volume)의 dataframe 생성
df = pd.read_csv('./AMZN.csv', index_col = 'Date', parse_dates = ['Date'])

# 결측치 확인
df[df.isin([np.nan, np.inf, -np.inf]).any(1)]

# 수정 종가를 슬라이싱해 서브 데이터셋 생성 및 그래프를 통한 모양 확인
price_df = df.loc[:, ['Adj Close']].copy()
price_df.plot(figsize=(16,9))

# 2000년대 초반 주가 폭락 부분 살펴보기 : 최대 낙폭(MDD, 최고점 대비 현재까지 하락한 비율 중 최대 하락율) 확인
from_date = '1997-01-03'
to_date = '2003-01-03'
price_df.loc[from_date : to_date].plot(figsize=(16,9))

# 일별 수익률 계산
price_df['daily_rtn'] = price_df['Adj Close'].pct_change()

# 매수한 시점부터 매도한 시점까지 일별 수익률을 누적 곱하여 종가 수익률을 계산한다 (바이앤홀드 전략수익률)
# cumprod() : 배열에서 주어진 축에 따라 누적되는 원소들의 누적 곱을 계산하는 함수
price_df['st_rtn'] = (1 + price_df['daily_rtn']).cumprod()
price_df['st_rtn'].plot(figsize=(16,9))

# 다른 시점을 기준으로 수익률 계산해보기
base_date = '2019-11-26'
tmp_df = price_df.loc[base_date:, ['st_rtn']] / price_df.loc[base_date, ['st_rtn']]
last_date = tmp_df.index[-1]
print('누적 수익 : ', tmp_df.loc[last_date, 'st_rtn'])
tmp_df.plot(figsize=(16,9))
```

> 바이앤홀드 전략에 매수/매도 신호를 추가한다면 다양한 전략으로 응용 가능

### 3.2 투자 성과 분석 지표
-------

* 위에서 살펴본 바이앤홀드 전략의 성과를 아래 4가지 지표로 분석
    * 연평균 복리 수익률(CAGR)
    * 최대 낙폭(MDD)
    * 변동성(Vol)
    * 샤프 지수

#### 3.2.1 연평균 복리 수익률(CAGR)
------

* 수익률 성과를 분석할 때는 복리 성질이 있는 기하평균을 사용한다.
* 연평균 복리 수익률 수식은 아래 코드로 나타낼 수 있다.
```python
# 데이터의 마지막 일자
last_date = '2020-11-24'
# 최종 누적 수익률의 누적 연도 제곱근 구하기 (1년을 252영업일로 계산)
CAGR = price_df.loc[last_date, 'st_rtn'] ** (252./len(price_df.index)) - 1
```

#### 3.2.2 최대 낙폭(MDD)
----

* 투자 기간 동안 고점부터 저점까지 떨어진 낙폭 중 최댓값을 의미함
* 투자자가 겪을 수 있는 최대 고통 지수.. 낮을수록 좋음
* MDD 계산 과정
```python
# cummax() : 누적 최대값, cummin() : 누적 최소값
historical_max = price_df['Adj Close'].cummax()
daily_drawdown = price_df['Adj Close'] / historical_max - 1.0
historical_dd = daily_drawdown.cummin()
historical_dd.plot()
```

#### 3.2.3 변동성(Vol)
--------

* 변동성에는 여러 종류가 있으며, 여기서는 주가 변화 수익률 관점의 변동성 확인
* 금융 자산의 방향성에 대한 불확실성과 가격 등락에 대한 위험 예상 지표로 해석
* 수익률의 표준 편차를 변동성으로 계산
* 변동성 수식을 코드로 나타내면 다음과 같다.
```python
# 1년의 영업일은 252일이라고 가정한다.
VOL = np.std(price_df['daily_rtn']) * np.sqrt(252.)
```
> 루이 바슐리에의 '투기이론'에 따르면 주가의 변동폭은 시간의 제곱근에 비례한다.

#### 3.2.4 샤프 지수
----------

* 위험 대비 수익성 지표
* 사후적 샤프 비율 = 실현 수익률의 산술평균 / 실현 수익률의 변동성
```python
Sharpe = np.mean(price_df['daily_rtn']) / VOL
```

#### 3.2.5 종합 성과 분석
---------

```python
CAGR = price_df.loc['2020-11-24', 'st_rtn'] ** (252./len(price_df.index)) - 1
MDD = (price_df['Adj Close'] / price_df['Adj Close'].cummax() - 1.0).cummin().min()
VOL = np.std(price_df['daily_rtn']) * np.sqrt(252.)
Sharpe = np.mean(price_df['daily_rtn']) / VOL

print('CAGR : ', round(CAGR * 100, 2), '%')
print('MDD : ', round(-1 * MDD * 100, 2), '%')
print('VOL : ', round(VOL * 100, 2), '%')
print('Sharpe : ', round(Sharpe, 2))
```
* 결과값 :
    * CAGR :  75.13 % → 상장 후 현재까지 연평균 복리 수익률 75% 성장
    * MDD :  22.74 % → 최대 낙폭은 22.74% 로 크지 않음
    * VOL :  38.41 % → 변동성은 38%로 주가 수익률이 해당 비율만큼 출렁임을 의미함
    * Sharpe :  0.01 → 특정 위험을 감수하는 만큼 수익률을 보장해주지 않는다 (1 이상이 되어야 좋음)
> 단, 해당 데이터는 2019-11-25 ~ 2020-11-24 까지의 1년치 데이터만 있으므로, 보다 정확한 분석을 위해서는 보다 많은 데이터가 필요<br/>
> 책의 경우에는 2000년대 주가 폭락 데이터도 포함되어 있기 때문에 위 결과와 상반되는 분석이 나왔다.